## 1. 문서의 내용을 읽는다.
- doc2txt를 사용해서 읽는다.

In [2]:
%pip install --upgrade --quiet docx2txt langchain-community

Note: you may need to restart the kernel to use updated packages.


## 2. 문서를 쪼갠다.
- 문서를 쪼개기 위해서 langchain-text-splitters를 사용해서 쪼갠다.

In [ ]:
%pip install -qU langchain-text-splitters

In [6]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,   
    chunk_overlap=200 
)

loader = Docx2txtLoader('./tax.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [7]:
len(document_list)

187

## 3. 임베딩
- 임베딩은 OpenAI를 이용해서 한다.
- 임베딩을 하면 백터 데이터베이스에 저장을 해야한다.
- Chroma는 인메모리 백터 데이터베이스 저장소이다.

In [22]:
%pip install langchain-chroma

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/611.1 kB ? eta -:--:--
   ---------------------------------------- 611.1/611.1 kB 7.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ----- ---------------------------------- 2.1/15.8 MB 9.8 MB/s eta 0:00:02
   ------- -------------------------------- 2.9/15.8 MB 6.5 MB/s eta 0:00:03
   --------- ------------------------------ 3.7/15.8 MB 5.9 MB/s eta 0:00:03
   ------------ --------------------------- 5.0/15.8 MB 5.8 MB/s eta 0:00:02
   ----------------- ---------------------- 6.8/15.8 MB 6.3 MB/s eta 0:00:02
   ---------------------- ----------------- 8.9/15.8 MB 6.8 MB/s

  You can safely remove it manually.
  You can safely remove it manually.


In [8]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

### Chroma를 persist 폴더를 사용하게 생성

In [38]:
from langchain_chroma import Chroma

database = Chroma.from_documents(documents=document_list,
                                 embedding=embedding,
                                 persist_directory="./chroma",
                                 collection_name='chroma-tax')

In [19]:
from langchain_chroma import Chroma

database = Chroma(collection_name='chorma-tax',
                  persist_directory="./chroma",
                  embedding_function=embedding)


In [ ]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
# retrieved_docs = database.similarity_search(query, k=3)

In [40]:
retrieved_docs

[Document(id='32dec8f7-07a1-48b6-adb1-ca002d799b94', metadata={'source': './tax.docx'}, page_content='1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대소득은 제외한다) 또는 해당 과세기간에 대통령령으로 정하는 총수입금액의 합계액이 2천만원 이하인 자의 주택임대소득(2018년 12월 31일 이전에 끝나는 과세기간까지 발생하는 소득으로 한정한다). 이 경우 주택 수의 계산 및 주택임대소득의 산정 등 필요한 사항은 대통령령으로 정한다.\n\n다. 대통령령으로 정하는 농어가부업소득\n\n라. 대통령령으로 정하는 전통주의 제조에서 발생하는 소득\n\n마. 조림기간 5년 이상인 임지(林地)의 임목(林木)의 벌채 또는 양도로 발생하는 소득으로서 연 600만원 이하의 금액. 이 경우 조림기간 및 세액의 계산 등 필요한 사항은 대통령령으로 정한다.\n\n바. 대통령령으로 정하는 작물재배업에서 발생하는 소득\n\n사. 대통령령으로 정하는 어로어업 또는 양식어업에서 발생하는 소득\n\n3. 근로소득과 퇴직소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 대통령령으로 정하는 복무 중인 병(兵)이 받는 급여\n\n나. 법률에 따라 동원된 사람이 그 동원 직장에서 받는 급여\n\n다. 「산업재해보상보험법」에 따라 수급권자가 받는 요양급여, 휴업급여, 장해급여, 간병급여, 유족급여, 유족특별급여, 장해특별급여, 장의비 또는 근로의 제공으로 인한 부상ㆍ질병ㆍ사망과 관련하여 근로자나 그 유족이 받는 배상ㆍ보상 또는 위자(慰藉)의 성질이 있는 급여\n\n라. 「근로기준법」 또는 「선원법」에 따라 근로자ㆍ선원 및 그 유족이 받는 요양보상금, 휴업보상금

In [27]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다
- [Context]를 참고해서 사용자의 질문에 답변해주세요

[Context]
{retrieved_docs}

Question: {query}
"""

ai_message = llm.invoke(prompt)

ai_message.content

'연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 기본적인 소득세율과 공제 항목을 고려해야 합니다. 기본적으로 한국의 소득세는 누진세율 구조를 가지고 있어 소득 구간에 따라 세율이 달라집니다. \n\n2023년 기준으로 연봉 5천만원인 경우를 살펴보면, 기본적으로 소득세율은 아래와 같이 적용됩니다:\n\n- 1,200만원 이하: 6%\n- 1,200만원 초과 ~ 4,600만원 이하: 15%\n- 4,600만원 초과 ~ 8,800만원 이하: 24%\n- 그 이상: 추가 세율 적용\n\n이를 바탕으로, 기본적인 소득세 계산을 대략적으로 해보면 다음과 같습니다:\n\n1. 1,200만원까지: \\(1,200만원 \\times 6\\% = 72만원\\)\n2. 1,200만원 초과 ~ 4,600만원: \\((4,600만원 - 1,200만원) \\times 15\\% = 510만원\\)\n3. 4,600만원 초과 ~ 5,000만원: \\((5,000만원 - 4,600만원) \\times 24\\% = 96만원\\)\n\n각 구간의 세금을 더하면:\n\\[72만원 + 510만원 + 96만원 = 678만원\\]\n\n따라서, 기본적으로 연봉 5천만원인 직장인의 소득세는 약 678만원입니다. 하지만, 실제로는 인적공제, 근로소득공제, 기부금 공제 등 다양한 공제가 적용될 수 있어 최종적으로 납부해야 할 소득세는 더 적어질 수 있습니다. 정확한 금액을 계산하기 위해서는 개별적인 공제 항목과 세액공제 등을 고려해야 합니다. 세무 전문가와 상담하거나 국세청의 홈택스 서비스를 통해 상세한 계산을 진행해 볼 수 있습니다.'

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

ai_message = llm.invoke(prompt)

ai_message.content

## LangChain은 체인인데 Retieval을 조금 더 효과적으로 하는 방법
- RetrievalQA Chain

In [46]:
%pip install -U langchain langchainhub --quiet

Note: you may need to restart the kernel to use updated packages.


In [29]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

c:\Users\jaedeok\study\llm-application\.venv\Lib\site-packages\langsmith\client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [30]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [31]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [32]:
ai_message = qa_chain({"query": query})

In [33]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '소득세 계산은 여러 요인에 따라 달라질 수 있으며, 누진세율이 적용되기 때문에 단순히 연봉만으로 소득세를 정확히 계산하기 어렵습니다. 구체적인 세금 계산에는 기본공제, 특별공제, 각종 세액공제 적용이 필요합니다. 정확한 소득세 금액은 국세청의 소득세 계산기 등을 통해 확인하는 것이 좋습니다.'}